### Import Required Libraries

In [21]:
import pandas as pd
import numpy as np
import re
import nltk
import spacy
from tqdm import tqdm

pd.set_option('display.max_colwidth', 200)

### Read Dataset

In [22]:
# read data
df_questions = pd.read_hdf('auto_tagging_data_v2.h5')

In [23]:
df_questions.head()

,Id,Title,Body,Tags
0,6,The Two Cultures: statistics vs. machine learning?,"<p>Last year, I read a blog post from <a href=""http://anyall.org/"">Brendan O'Connor</a> entitled <a href=""http://anyall.org/blog/2008/12/statistics-vs-machine-learning-fight/"">""Statistics vs. Mach...",[machine-learning]
1,21,Forecasting demographic census,<p>What are some of the ways to forecast demographic census with some validation and calibration techniques?</p>\n\n<p>Some of the concerns:</p>\n\n<ul>\n<li>Census blocks vary in sizes as rural\n...,[forecasting]
2,22,Bayesian and frequentist reasoning in plain English,<p>How would you describe in plain English the characteristics that distinguish Bayesian from Frequentist reasoning?</p>\n,[bayesian]
3,31,What is the meaning of p values and t values in statistical tests?,"<p>After taking a statistics course and then trying to help fellow students, I noticed one subject that inspires much head-desk banging is interpreting the results of statistical hypothesis tests....","[hypothesis-testing, t-test, p-value, interpretation]"
4,36,Examples for teaching: Correlation does not mean causation,"<p>There is an old saying: ""Correlation does not mean causation"". When I teach, I tend to use the following standard examples to illustrate this point:</p>\n\n<ol>\n<li>number of storks and birth ...",[correlation]


In [24]:
# combine title and body
df_questions['Text'] = df_questions["Title"] + " " + df_questions["Body"]

### Clean and Pre-process Data

In [25]:
def clean_text(text):
    # remove html tags and url links
    text = re.sub(r'<.*?>', '', text)
    # remove everything alphabets
    text = re.sub("[^a-zA-Z]"," ",text)
    # remove whitespaces
    text = ' '.join(text.split())
    
    return text

In [26]:
df_questions['Text'] = df_questions['Text'].apply(lambda x: clean_text(x))
df_questions['Text'] = df_questions['Text'].str.lower()

In [27]:
df_questions[['Id', 'Text', 'Tags']].sample(5)

,Id,Text,Tags
43187,10526,canonical correlation analysis and time series analysis is there a way to utilize canonical correlation analysis when your data are time series and repeated measures i e your experimental units ar...,"[time-series, correlation, multivariate-analysis]"
32755,88819,mcmc methods burning samples in mcmc methods i keep reading about burn in time or the number of samples to burn what is this exactly and why is it needed update once mcmc stabilizes does it remain...,"[sampling, mcmc]"
73868,25908,how to determine the number of clusters when using correlation as the distance how does using correlation as the distance influence the determination of the number of clusters when doing kmeans is...,"[r, correlation, clustering]"
73742,167829,how do we decide on how to fill missing values in data i have a data set with na values in many predictor variables how do we impute the best values i have variables in total out of them belong to...,[regression]
17349,159496,testing for differences between populations of individuals with binomial data so i am trying to compare between populuations and determine whether there is a difference in count based data my data...,"[hypothesis-testing, statistical-significance, anova, binomial]"


### Encode Text to Numbers

In [28]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [29]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_questions['Text'])

In [30]:
# check unique words count
len(tokenizer.word_index)

81956

In [31]:
# check unique words count
vocab_size = len(tokenizer.word_index) + 1
vocab_size

81957

In [32]:
sequences = tokenizer.texts_to_sequences(df_questions['Text'])

In [33]:
i = 0
print(df_questions['Text'][i], '\n'), print(sequences[i])

the two cultures statistics vs machine learning last year i read a blog post from brendan o connor entitled statistics vs machine learning fight that discussed some of the differences between the two fields andrew gelman responded favorably to this simon blomberg from r s fortunes package to paraphrase provocatively machine learning is statistics minus any checking of models and assumptions brian d ripley about the difference between machine learning and statistics user vienna may season s greetings andrew gelman in that case maybe we should get rid of checking of models and assumptions more often then maybe we d be able to solve some of the problems that the machine learning people can solve but we can t there was also the statistical modeling the two cultures paper by leo breiman in which argued that statisticians rely too heavily on data modeling and that machine learning techniques are making progress by instead relying on the predictive accuracy of models has the statistics field 

(None, None)

__Find right maximum length for the sequences__

In [34]:
seq_lengths = []

for i in sequences:
    seq_lengths.append(len(i))

In [35]:
print("30th percentile: ", pd.Series(seq_lengths).quantile(0.3))
print("40th percentile: ", pd.Series(seq_lengths).quantile(0.4))
print("50th percentile: ", pd.Series(seq_lengths).quantile(0.5))
print("60th percentile: ", pd.Series(seq_lengths).quantile(0.6))
print("70th percentile: ", pd.Series(seq_lengths).quantile(0.7))
print("80th percentile: ", pd.Series(seq_lengths).quantile(0.8))
print("90th percentile: ", pd.Series(seq_lengths).quantile(0.9))
print("95th percentile: ", pd.Series(seq_lengths).quantile(0.95))
print("99th percentile: ", pd.Series(seq_lengths).quantile(0.99))

30th percentile:  97.0
40th percentile:  116.0
50th percentile:  137.0
60th percentile:  162.0
70th percentile:  193.0
80th percentile:  238.0
90th percentile:  320.0
95th percentile:  411.0
99th percentile:  678.0


In [36]:
max_length = 125

# padding
padded_seq = pad_sequences(sequences, maxlen=max_length)

### Reshape Target Variable

In [37]:
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_questions['Tags'])
y = multilabel_binarizer.transform(df_questions['Tags'])

In [38]:
padded_seq.shape, y.shape

((76365, 125), (76365, 100))

### Train-Test Split

In [39]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(padded_seq, y, test_size=0.2, random_state=9)

### Model Building

In [40]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, GlobalMaxPool1D, Dropout, Conv1D
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [46]:
model = Sequential()
model.add(Embedding(vocab_size+1, 128, input_length = max_length))
model.add(Dropout(0.15))
model.add(Conv1D(300, 3, padding = 'valid', activation = "relu", strides = 1))
model.add(GlobalMaxPool1D())
model.add(Dense(100, activation = "sigmoid"))
#model.add(Activation('sigmoid'))

In [47]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 125, 128)          10490624  
                                                                 
 dropout_2 (Dropout)         (None, 125, 128)          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 123, 300)          115500    
                                                                 
 global_max_pooling1d_2 (Glo  (None, 300)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_2 (Dense)             (None, 100)               30100     
                                                                 
Total params: 10,636,224
Trainable params: 10,636,224
Non-trainable params: 0
__________________________________________

In [48]:
callbacks = [EarlyStopping(patience=3),ModelCheckpoint(filepath='model-conv1d_v1.h5', save_best_only=True)]

In [49]:
# train model
history = model.fit(x_train, y_train,epochs=10,batch_size=512,validation_split=0.1,callbacks=callbacks)

Epoch 1/10
108/108 [==============================] - 93s 852ms/step - loss: 0.1858 - accuracy: 0.0473 - val_loss: 0.0905 - val_accuracy: 0.0615
Epoch 2/10
108/108 [==============================] - 86s 794ms/step - loss: 0.0893 - accuracy: 0.0674 - val_loss: 0.0871 - val_accuracy: 0.0851
Epoch 3/10
108/108 [==============================] - 87s 806ms/step - loss: 0.0836 - accuracy: 0.1163 - val_loss: 0.0800 - val_accuracy: 0.1390
Epoch 4/10
108/108 [==============================] - 84s 780ms/step - loss: 0.0749 - accuracy: 0.1987 - val_loss: 0.0706 - val_accuracy: 0.2439
Epoch 5/10
108/108 [==============================] - 85s 785ms/step - loss: 0.0671 - accuracy: 0.2633 - val_loss: 0.0652 - val_accuracy: 0.2787
Epoch 6/10
108/108 [==============================] - 84s 779ms/step - loss: 0.0621 - accuracy: 0.2944 - val_loss: 0.0612 - val_accuracy: 0.3011
Epoch 7/10
108/108 [==============================] - 84s 776ms/step - loss: 0.0584 - accuracy: 0.3150 - val_loss: 0.0586 - val_ac

### Predictions and Performane Evaluation

In [50]:
# use the code below to load the saved model
# model = load_model('model-conv1d_v1.h5')

In [51]:
preds = model.predict(x_val)

478/478 [==============================] - 7s 14ms/step


In [52]:
preds.shape

(15273, 100)

In [53]:
# set threshold to 0.45
preds_int = (preds >= 0.45).astype(int)

In [54]:
from sklearn.metrics import f1_score

# calculate F1 score
f1_score(y_val, preds_int, average="micro")

0.47293110476966976

### Inference

In [55]:
# model = load_model('model-conv1d_v1.h5')

In [58]:
def infer_tags(q):
    q = clean_text(q)
    q = q.lower()
    q_seq = tokenizer.texts_to_sequences([q])
    q_seq_padded = pad_sequences(q_seq, maxlen=300)
    q_pred = model.predict(q_seq_padded)
    q_pred = (q_pred >= 0.3).astype(int)
    return multilabel_binarizer.inverse_transform(q_pred)

In [69]:
# give new question
new_q = "Regression line in ggplot doesn't match computed regression Im using R and created a chart using ggplot2. I then create a regression so I can make some predicitions I pass my data frame of to the predict function predict(regression, Measures) I'd expect the predictions to be the same as if I used the regression line on the chart, but they aren't the same. Why would this be the case? Is there a setting in ggplot or is my expectation incorrect?"

newq= "Including -Inf on ggplot2 axis"
# get tags
infer_tags(newq)

1/1 [==============================] - 0s 279ms/step


[('data-visualization', 'r')]